# Language modelling

In this tutorial we will use two character level language models to create dinosaur names. We will recurrent neural networks as main tool for language modellig. 

![picture](https://vignette.wikia.nocookie.net/uncyclopedia/images/7/73/Hankk_the_dino.png/revision/latest?cb=20100127020302)

In [8]:
!wget https://raw.githubusercontent.com/artemovae/NLP-seminar-LM/master/dinos.txt

/bin/sh: wget: command not found


## Recurrent neural networks

Input:

$x_{1:n} = x_1, x_2, \ldots, x_n$, $x_i \in \mathbb{R}^{d_{in}}$

For each input $x_{1:i}$ we get an output $y_i$:

$y_i = RNN(x_{1:i})$, $y_i \in \mathbb{R}^{d_{out}}$

For the whole input sequence $x_{1:n}$:

$y_{1:n} = RNN^{*}(x_{1:n})$, $y_i \in \mathbb{R}^{d_{out}}$

$R$ is a recursive activation function with two inputs: $x_i$ и $s_{i-1}$ (state vector)

$RNN^{*}(x_{1:n}, s_0) = y_{1:n}$

$y_i = O(s_i) = g(W^{out}[s_{i} ,x_i] +b)$

$s_i = R(s_{i-1}, x_i)$

$s_i = R(s_{i-1}, x_i) = g(W^{hid}[s_{i-1} ,x_i] +b)$  -- concatenate $[s_{i-1}, x]$

$x_i \in \mathbb{R}^{d_{in}}$, $y_i \in \mathbb{R}^{ d_{out}}$, $s_i \in \mathbb{R}^{d_{hid}}$

$W^{hid} \in \mathbb{R}^{(d_{in}+d_{out}) \times d_{hid}}$, $W^{out} \in \mathbb{R}^{d_{hid} \times d_{out}}$

![rnn](https://github.com/enggen/Deep-Learning-Coursera/raw/1407e19c98833d2686a0748db26b594f3102301e/Sequence%20Models/Week1/Dinosaur%20Island%20--%20Character-level%20language%20model/images/rnn.png)

We are going to create an RNN-LM using pytorch

In [9]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import pdb
from torch.utils.data import Dataset, DataLoader

%load_ext autoreload
%autoreload 2

torch.set_printoptions(linewidth=200)

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
hidden_size = 50

print(device)

cpu


**Step 1**. Prepare the dataset

In [11]:
class DinosDataset(Dataset):
    def __init__(self):
        super().__init__()
        with open('dinos.txt') as f:
            content = f.read().lower()
            self.vocab = sorted(set(content)) + ['<', '>']
            self.vocab_size = len(self.vocab)
            self.lines = content.splitlines()
        self.ch_to_idx = {c:i for i, c in enumerate(self.vocab)}
        self.idx_to_ch = {i:c for i, c in enumerate(self.vocab)}
    
    def __getitem__(self, index):
        line = self.lines[index]
        #teacher forcing
        x_str = '<' + line 
        y_str = line + '>' 
        x = torch.zeros([len(x_str), self.vocab_size], dtype=torch.float)
        y = torch.empty(len(x_str), dtype=torch.long)
        for i, (x_ch, y_ch) in enumerate(zip(x_str, y_str)):
            x[i][self.ch_to_idx[x_ch]] = 1
            y[i] = self.ch_to_idx[y_ch]
        
        return x, y
    
    def __len__(self):
        return len(self.lines)

In [12]:
trn_ds = DinosDataset()
trn_dl = DataLoader(trn_ds, shuffle=True)

In [13]:
trn_ds.lines[1]

'aardonyx'

In [14]:
print(trn_ds.idx_to_ch)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 27: '<', 28: '>'}


In [15]:
trn_ds.vocab_size

29

In [16]:
x, y = trn_ds[1]

In [17]:
x.shape

torch.Size([9, 29])

In [18]:
y.shape

torch.Size([9])

**Step 2**. Define a model, loss function and optimization algorithm

In [19]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.i2h = nn.Linear( <your code here>)
        self.dropout = nn.Dropout(0.3)
        self.i2o = nn.Linear( <your code here>)
    
    def forward(self, h_prev, x):
        return <your code here>

In [20]:
model = RNN(trn_ds.vocab_size, hidden_size, trn_ds.vocab_size).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2)

**Step 3**. Declare a sampling procedure

![rnn](https://github.com/enggen/Deep-Learning-Coursera/raw/1407e19c98833d2686a0748db26b594f3102301e/Sequence%20Models/Week1/Dinosaur%20Island%20--%20Character-level%20language%20model/images/dinos3.png)

Sidenote: we can use Softmax probabilities to evaluate perplexety, which a standart quality measure for the task

$ 2^{{H(p)}}=2^{{-\sum _{x}p(\hat(y)^{<i>} == y^{<i>}_{train} )\log _{2}p(\hat(y)^{<i>} == y^{<i>}_{train})}} $

In [21]:
def sample(model):
    model.eval()
    word_size=0
    newline_idx = trn_ds.ch_to_idx['>']
    with torch.no_grad():
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x = h_prev.new_zeros([1, trn_ds.vocab_size])
        start_char_idx = trn_ds.ch_to_idx['<']
        indices = [start_char_idx]
        x[0, start_char_idx] = 1
        predicted_char_idx = start_char_idx
        
        while predicted_char_idx != newline_idx and word_size != 50:
            h_prev, y_pred = model(h_prev, x)
            y_softmax_scores = torch.softmax(y_pred, dim=1)
            
            np.random.seed(np.random.randint(1, 5000))
            idx = np.random.choice(np.arange(trn_ds.vocab_size), p=y_softmax_scores.cpu().numpy().ravel())
            indices.append(idx)
            
            x = (y_pred == y_pred.max(1)[0]).float()
 
            
            predicted_char_idx = idx
            
            word_size += 1
        
        if word_size == 50:
            indices.append(newline_idx)
    return indices

In [22]:
def print_sample(sample_idxs):
    [print(trn_ds.idx_to_ch[x], end ='') for x in sample_idxs]
    print()

**Step 4**. Almost done! Train the model

In [32]:
def train_one_epoch(model, loss_fn, optimizer):
    model.train()
    for line_num, (x, y) in enumerate(trn_dl):
        loss = 0
        optimizer.zero_grad()
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x, y = x.to(device), y.to(device)
        for i in range(x.shape[1]):
            h_prev, y_pred = model(h_prev, x[:, i])
            loss += loss_fn(y_pred, y[:, i])
            
        if (line_num+1) % 100 == 0:
            print_sample(sample(model))
        loss.backward()
        optimizer.step()

In [33]:
def train(model, loss_fn, optimizer, dataset='dinos', epochs=1):
    for e in range(1, epochs+1):
        print('Epoch:{}'.format(e))
        train_one_epoch(model, loss_fn, optimizer)
        print()

In [37]:
train(model, loss_fn, optimizer, epochs = 5)

Epoch:1
<aunuaauras>
<tmrasausus>
<suroiturus>
<rusasaerus>
<gubestarus>
<icsas>
<puattorastp>
<tqrusauras>
<aurubauras>
<tlrasaos>
<uerus>
<duriyrps>
<tapnbauius>
<ahroaoes>
<hcras>

Epoch:2
<puiussurus>
<zurusuarup>
<scrmtanopdor>
<gnbucaurus>
<turesnusus>
<suapsanras>
<taiusaurus>
<aucnsnuous>
<llrur>
<srrusauras>
<aumuaaures>
<sancucaurus>
<ttilosaurus>
<tarocaulus>
<ahrnbamsopaurus>

Epoch:3
<uhrusaurus>
<sprusaurus>
<aurucauris>
<smnasaurus>
<suroiourus>
<purasderus>
<fuaitocem>
<gmbucaurus>
<turasiusus>
<suainaurts>
<tchspaurus>
<aubnteurus>
<llrrpyurus>
<aprapoctosic>
<larhsaueur>

Epoch:4
<xrsasosooeurus>
<putcoodateor>
<agpnaasrus>
<tdruptirus>
<durotturusaurasris>
<tceusaurus>
<auaotpurus>
<llrur>
<sprosantap>
<aupucauras>
<tmrasaurus>
<surorturus>
<purareerus>
<fuabsacros>
<hcranturus>

Epoch:5
<srrusius>
<purarakrus>
<euaetocaurus>
<tcivotaxtsaurus>
<snrusaurss>
<aubtaecuhus>
<gmbuchytrl>
<turistot>
<ltrubgocaurus>
<agrpaoltpterui>
<pucusttrus>
<zurustcrus>
<scrisanrui>
<c

### Tasks 
1. Complete the forward fuction for the RNN model
2. Rewrite the sampling function so that pangrams (words that contain each character only ones) are generated

### Advanced tasks
3. Make yourself a dinosaur nickname: initialize the sampling procedure with your own name
4. Implemet a fuction ```get_perplexity``` to compute perplexety. Add more layers and check, whether they affect the perplexety.

# Reference

1. Sampling in  RNN: https://nlp.stanford.edu/blog/maximum-likelihood-decoding-with-rnns-the-good-the-bad-and-the-ugly/
2. Coursera course (main source): https://github.com/furkanu/deeplearning.ai-pytorch/tree/master/5-%20Sequence%20Models
3. Coursera course (main source): https://github.com/Kulbear/deep-learning-coursera/blob/master/Sequence%20Models/Dinosaurus%20Island%20--%20Character%20level%20language%20model%20final%20-%20v3.ipynb
4. LSTM: http://colah.github.io/posts/2015-08-Understanding-LSTMs/